# Database

In this file the dataset is created in which the graph data will be stored. First part is to read the *.dem* files of the matches played on inferno, than convert the datas into ***HeteroData*** structure so that it can be stored in a PyG database.

## Creating HeteroData objects from game snapshots

In [1]:
import torch
from torch_geometric.data import HeteroData, DataLoader
from torch_geometric.data import Dataset, Data

import matplotlib.pyplot as plt
from math import floor
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

Reading the inferno csv files

In [2]:
def get_inferno_matches(folder_path):
    matches = []
    for filename in os.listdir(folder_path + '/frames/'):
        if 'inferno' in filename:
            matches.append(filename)
    return matches

Reading dataframes.

In [3]:
def get_base_dataframes(folder_path, filename):
    playerFrames = pd.read_csv(folder_path + '/playerFrames/' + filename)
    kills = pd.read_csv(folder_path +'/kills/' + filename)
    rounds = pd.read_csv(folder_path +'/rounds/' + filename)


    rounds = rounds[['roundNum', 'tScore', "ctScore" ,'endTScore', 'endCTScore']]
    pf = playerFrames[['tick', 'roundNum', 'seconds', 'side', 'name', 'x', 'y', 'z','eyeX', 'eyeY', 'eyeZ', 'velocityX', 'velocityY', 'velocityZ',
        'hp', 'armor', 'activeWeapon','flashGrenades', 'smokeGrenades', 'heGrenades', 'totalUtility','isAlive', 'isReloading', 'isDefusing', 'isPlanting', 'isUnknown',
        'equipmentValue', 'equipmentValueRoundStart', 'hasHelmet','hasDefuse', 'hasBomb']]
    
    return pf, kills, rounds

Formatting the dataframes.

In [4]:
def format_base_dataframes(pf, kills, rounds):
    
    # Merge playerFrames with rounds
    pf = pf.merge(rounds, on='roundNum')
    
    # Calculate whether or not a player won the round
    pf["winsRounds"] = pf.apply(lambda x: 
        1 if ( (x['side'] == 'T') and (x['endTScore'] > x['tScore']) ) or ( (x['side'] == 'CT') and (x['endCTScore'] > x['ctScore']) )
        else 0, axis=1)

    pf['kills'] = 0
    pf['deaths'] = 0
    
    # Setting kill-counts
    for index, row in kills.iterrows():
        pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'kills'] += 1
        pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['victimName']), 'deaths'] += 1
        
    # Rounded-down seconds
    pf['sec'] = pf['seconds'].apply(lambda x: floor(x))
        
    return pf, kills, rounds

In [5]:
def get_dummies(pf):
    
    # Create dummie cols
    dummies = pd.get_dummies(pf['activeWeapon'], prefix="activeWeapon",drop_first=False)[['activeWeapon_AK-47','activeWeapon_Knife','activeWeapon_AWP','activeWeapon_M4A1','activeWeapon_Smoke Grenade','activeWeapon_M4A4','activeWeapon_Galil AR','activeWeapon_Desert Eagle','activeWeapon_Flashbang','activeWeapon_Glock-18','activeWeapon_USP-S']]
    dummies = dummies*1
    pf = pf.merge(dummies, left_index = True, right_index = True, how = 'left')
    
    return pf

In [6]:
def create_players_dataframes(pf):
    
    startAsCTPlayerNames = pf[(pf['side'] == 'CT') & (pf['roundNum'] == 1)]['name'].unique()
    startAsTPlayerNames = pf[(pf['side'] == 'T') & (pf['roundNum'] == 1)]['name'].unique()
    players = {}

    # Team 1: start on CT side
    players[0] = pf[pf['name'] == startAsCTPlayerNames[0]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[1] = pf[pf['name'] == startAsCTPlayerNames[1]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[2] = pf[pf['name'] == startAsCTPlayerNames[2]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[3] = pf[pf['name'] == startAsCTPlayerNames[3]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[4] = pf[pf['name'] == startAsCTPlayerNames[4]].drop_duplicates(subset=['sec','roundNum']).copy()

    # Team 2: start on T side
    players[5] = pf[pf['name'] == startAsTPlayerNames[0]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[6] = pf[pf['name'] == startAsTPlayerNames[1]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[7] = pf[pf['name'] == startAsTPlayerNames[2]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[8] = pf[pf['name'] == startAsTPlayerNames[3]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[9] = pf[pf['name'] == startAsTPlayerNames[4]].drop_duplicates(subset=['sec','roundNum']).copy()
    
    return players


In [7]:
# Calculate closest map-graph node
def find_closest(row):
    nodes = pd.read_csv('../inferno_graph/graph_models/manual/nodes_v1_5.csv')
    distances = np.sqrt((nodes['x'] - row['x'])**2 + (nodes['y'] - row['y'])**2)
    return nodes.loc[distances.idxmin(), 'nodeId']

def calculate_closest_map_node_to_player(pf):
    
    # Read nodes and create players
    players = create_players_dataframes(pf)

    for idx, player in enumerate(players):
        
        players[idx] = players[idx].replace(True, 1)
        players[idx] = players[idx].replace(False, 0)
        players[idx]['isCT'] = players[idx]['side'].apply(lambda x: 1 if x == 'CT' else 0)
        players[idx]['closestId'] = players[idx].apply(find_closest, axis=1)

        del players[idx]['side']
        del players[idx]['activeWeapon']
        del players[idx]['winsRounds']
    
    return players

In [8]:
def get_player_edges(players):
    
    playerEdges = None

    for idx in range(0,len(players)):
        temp = players[idx][['roundNum','sec','closestId']].copy()
        temp['playerId'] = idx
        if playerEdges is None:
            playerEdges = temp
        else:
            playerEdges = pd.concat([playerEdges, temp])
    
    return playerEdges

In [9]:
def get_graph_level_data(players, rounds):
    # Copy players object
    graph_players = {}
    for idx in range(0,len(players)):
        graph_players[idx] = players[idx].copy()

    colsNotToRename = ['tick', 'roundNum', 'seconds', 'sec']

    # Rename columns except for tick, roundNum, seconds, floorSec
    for idx in range(0,len(graph_players)):
        
        for col in graph_players[idx].columns:
            if col not in colsNotToRename:
                graph_players[idx].rename(columns={col: "player" + str(idx) + "_" + col}, inplace=True)

    # Create a graph dataframe to store all players in 1 row per second
    graph_data = graph_players[0].copy()

    # Merge dataframes
    for i in range(1, len(graph_players)):
        graph_data = graph_data.merge(graph_players[i], on=colsNotToRename)
        
    graph_data = graph_data.merge(rounds, on=['roundNum'])
    graph_data['CTwinsRound'] = graph_data.apply(lambda x: 
        1 if 
        (x['endCTScore'] > x['ctScore'])
        else 0, axis=1)
    graph_data['team1AliveNum'] = graph_data[['player0_isAlive','player1_isAlive','player2_isAlive','player3_isAlive','player4_isAlive']].sum(axis=1)
    graph_data['team2AliveNum'] = graph_data[['player5_isAlive','player6_isAlive','player7_isAlive','player8_isAlive','player9_isAlive']].sum(axis=1)
    graph_data = graph_data[['roundNum','seconds','sec','team1AliveNum','team2AliveNum','CTwinsRound']]
    
    return graph_data

In [10]:
def get_nodes_and_edges():
    nodes = pd.read_csv('../inferno_graph/graph_models/manual/nodes_v1_5.csv')
    edges = pd.read_csv('../inferno_graph/graph_models/manual/edges_v1_5.csv')
    
    nodes['x'] = nodes['x'].astype('float32')
    nodes['y'] = nodes['y'].astype('float32')
    nodes['isPath'] = nodes['type'].apply(lambda x: 1 if x == 'path' else 0)
    
    return nodes, edges

In [11]:
def create_playerFrameData_for_graph(players, roundNum, sec):
    playerFrameData = pd.DataFrame(columns=players[0].columns)
    for idx in range(0,len(players)):
        playerFrameData.loc[idx] = players[idx].loc[(players[idx]['roundNum'] == roundNum) & (players[idx]['sec'] == sec)].iloc[0]
        
    playerFrameData.drop(columns=['tick', 'roundNum', 'seconds', 'name', 'closestId'], inplace=True)
    playerFrameData = playerFrameData.astype('float32')
    
    return playerFrameData

In [12]:
def create_HeteroData(playerFrameData, playerEdges, graph_data, nodes, edges, roundNum, sec):
    data = HeteroData()

    # Create a sample heterogeneous graph with node, edge, and multiple graph-level features
    data['player'].x = torch.tensor(playerFrameData.astype('float32').values)
    data['map'].x = torch.tensor(nodes[['x','y','isPath']].astype('float32').values)

    data['map', 'connected_to', 'map'].edge_index = torch.tensor(edges.T.values)
    data['player', 'closest_to', 'map'].edge_index = torch.tensor(playerEdges.loc[ (playerEdges['roundNum'] == roundNum) & (playerEdges['sec'] == sec)][['playerId','closestId']].T.values)


    # Define multiple graph-level features
    data.y = {
        'roundNum': roundNum.astype('float32'),
        'sec': sec.astype('float32'),
        'team1AliveNum': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['team1AliveNum'].iloc[0].astype('float32')),
        'team2AliveNum': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['team2AliveNum'].iloc[0].astype('float32')),
        'CTwinsRound': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['CTwinsRound'].iloc[0])
    }
    
    return data

Code to create the HeteroData object.

```python

## Dataset for HeteroData objects

In [13]:
class InfernoDataset(Dataset):
    def __init__(self, data_list):
        super(InfernoDataset, self).__init__()
        self.data_list = data_list

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]


Codes testing the created dataset class

In [15]:
# infdataset = InfernoDataset([data])

# torch.save(infdataset, './data/_/test_dataset.pt')

# dataset = torch.load('./data/_/test_dataset.pt')

# loader = DataLoader(infdataset, batch_size=1, shuffle=True)

## Creating the dataset

In [16]:
folder_path = '../scrape-parse/demo/parse/2023'
matches = get_inferno_matches(folder_path)
hetero_data_list = []

for idx,match in enumerate(matches):
    pf, kills, rounds = get_base_dataframes(folder_path, match)
    pf, kills, rounds = format_base_dataframes(pf, kills, rounds)
    pf = get_dummies(pf)
    players = calculate_closest_map_node_to_player(pf)
    player_edges = get_player_edges(players)
    graph_data = get_graph_level_data(players, rounds)
    nodes, edges = get_nodes_and_edges()

    rounds_list = pf['roundNum'].unique()
    for roundNum in rounds_list:
        sec_list = pf[pf['roundNum'] == roundNum]['sec'].unique()
        for sec in sec_list:
            playerFrameData = create_playerFrameData_for_graph(players, roundNum, sec)
            data = create_HeteroData(playerFrameData, player_edges, graph_data, nodes, edges, roundNum, sec)
            hetero_data_list.append(data)
        print(match,roundNum)
    
    if idx % 5 == 0:
        infdataset = InfernoDataset(hetero_data_list)
        torch.save(infdataset, './data/inferno_graph_dataset.pt')
        hetero_data_list = []

blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 1
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 2
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 3
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 4
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 5
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 6
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 7
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-skJ60UDCIzjbZIG70cNfnjevil-geniuses-vs-g2-m1-inferno.dem.csv 8
blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-s

KeyboardInterrupt: 

In [66]:
infdataset = InfernoDataset(hetero_data_list)
torch.save(infdataset, './data/blast-premier-fall-groups-2023-evil-geniuses-vs-g2-bo3-m1-inferno.pt')